# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [2]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [5]:
from sklearn.externals import joblib
eclf = joblib.load('sentiment.pkl') 

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [6]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-03-22 18:59:06,094 - watson_machine_learning_client.wml_client_error - WARNING - Publishing model failed.
Reason: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Server': 'nginx', 'Date': 'Sun, 22 Mar 2020 21:59:06 GMT', 'Content-Type': 'application/json', 'Content-Length': '187', 'Connection': 'keep-alive', 'X-Frame-Options': 'DENY', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1', 'Pragma': 'no-cache', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'X-WML-User-Client': 'PythonClient', 'x-global-transaction-id': '6990n2m0249460q91a08c7058arn1b52cb2f', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains'})
HTTP response body: {"trace":"1ic0jnfxgrqkl","errors":[{"code":"invalid_framework_input","message":"The framework runtimes values specified: [{\"name\":\"python\",\"version\":\"3.7\"}] are not supported."}]}

2020-03-22 19:01:24,168 - watson_machine_learning_client.wml_client_error - WARNING - Failure during getting 

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [9]:
wml_credentials={
  "apikey": "qGvxzCu7Rp_oy7K8PlNHS7LNqO1rNiZjC09xMQaFbyc-",
  "instance_id": "e2624527-9f60-40da-9da6-637637601a99",
  "url": "https://us-south.ml.cloud.ibm.com"
}

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [10]:
client = WatsonMachineLearningAPIClient(wml_credentials)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [12]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Juli", 
               client.repository.ModelMetaNames.NAME: "Sistemas de Recomendacion"}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
pipeline = make_pipeline(vect, eclf)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [15]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token...
                                                             nesterovs_momentum=True,
                                                             power_t=0.5,
                                                             random_state=None,
                                                             s

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [18]:
import pickle
pickle.dump( pipeline, open( "sistema_de_recomendaciones.pkl", 'wb') )

In [19]:
!mkdir model-dir
!cp svd_reviews.pkl model-dir

In [20]:
!tar -zcvf sistema_de_recomendaciones.tar.gz sistema_de_recomendaciones.pkl

sistema_de_recomendaciones.pkl


In [22]:
metadata = {
    client.repository.ModelMetaNames.NAME: "Sistema de Recomendaciones",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "scikit-learn",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "0.19"
}
published_model = client.repository.store_model( model="sistema_de_recomendaciones.tar.gz", meta_props=metadata )


**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [23]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [24]:
models_details = client.repository.list_models()

------------------------------------  ---------------------------  ------------------------  -----------------
GUID                                  NAME                         CREATED                   FRAMEWORK
387bff9d-36a6-40bd-8faf-a7c6acf29fee  Sistema de Recomendaciones   2020-03-22T22:01:47.850Z  scikit-learn-0.19
c4d7a7f7-99d2-44f2-8fea-1d66da17f9ce  scikit-learn model (tar.gz)  2020-03-22T21:33:44.320Z  scikit-learn-0.19
ff74001d-7827-4fdd-ba94-1c4a28d76f91  scikit-learn model (tar.gz)  2020-03-22T21:31:52.441Z  scikit-learn-0.19
------------------------------------  ---------------------------  ------------------------  -----------------


**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [25]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [26]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

roc_auc score:  0.865
              precision    recall  f1-score   support

           0       0.83      0.92      0.87       200
           1       0.91      0.81      0.86       200

    accuracy                           0.86       400
   macro avg       0.87      0.86      0.86       400
weighted avg       0.87      0.86      0.86       400

